## Import packages

In [ ]:
!pip install sdv
!pip install sdmetrics
!pip install kmodes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 3.9 MB/s 
     |████████████████████████████████| 1.6 MB 62.8 MB/s 
     |████████████████████████████████| 61 kB 179 kB/s 
     |████████████████████████████████| 47 kB 2.5 MB/s 
     |████████████████████████████████| 53 kB 958 kB/s 
     |████████████████████████████████| 131 kB 42.4 MB/s 
     |████████████████████████████████| 9.4 MB 33.7 MB/s 
     |████████████████████████████████| 295 kB 79.3 MB/s 
     |████████████████████████████████| 965 kB 27.6 MB/s 
     |████████████████████████████████| 662 kB 61.0 MB/s 
     |████████████████████████████████| 280 kB 63.0 MB/s 
     |████████████████████████████████| 15.3 MB 708 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: psutil
    Found existing installation: ps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sdv.timeseries import PAR
from sklearn.preprocessing import LabelEncoder
import kmodes.kprototypes as kk
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preprocessing

In [ ]:
# https://data.world/codefordc/purchase-card-transactions
df = pd.read_csv('/content/drive/Shareddrives/SI670 Final_Proj/transactions.csv')
# df = pd.read_csv('transactions.csv')
df # 275127 rows × 7 columns

,OBJECTID,AGENCY,TRANSACTION_DATE,TRANSACTION_AMOUNT,VENDOR_NAME,VENDOR_STATE_PROVINCE,MCC_DESCRIPTION
0,481,Department of Youth Rehabilitation Services,2009-01-06T00:00:00.000Z,977.00,MY SAFE FOOD HANDLER,MD,Business Services Not Elsewhere Classified
1,482,Department of Youth Rehabilitation Services,2009-01-06T00:00:00.000Z,-25.61,AMES SAFETY ENVELOPE C,MA,Miscellaneous Publishing And Printing
2,483,DC Public Library,2009-01-07T00:00:00.000Z,1035.29,WWW.AIA.ORG,DC,"Membership Organizations, Not Elsewhere Classi..."
3,484,Office of the State Superintendent of Education,2009-01-06T00:00:00.000Z,222.40,DELTA 00675274330522,TN,Delta
4,485,Office of the State Superintendent of Education,2009-01-06T00:00:00.000Z,222.40,DELTA 00675274330500,TN,Delta
...,...,...,...,...,...,...,...
275122,275123,Department of Human Services,2016-12-16T00:00:00.000Z,1707.33,STANDARD OFFICE SUPPLY,DC,"Stationery, Office & School Supply Stores"
275123,275124,Department of Youth Rehabilitation Services,2016-12-16T00:00:00.000Z,834.25,GLOBAL VISION TECHNOLO,MO,"Computers,Computer Peripheral Equipment, Software"
275124,275125,Office of Latino Affairs,2016-12-17T00:00:00.000Z,181.50,AMAZON MKTPLACE PMTS,WA,Book Stores
275125,275126,Office of Latino Affairs,2016-12-19T00:00:00.000Z,129.90,AMAZON MKTPLACE PMTS,WA,Book Stores


In [ ]:
# df.OBJECTID.value_counts() # 275127 OBJECTID
df.drop(['OBJECTID'], axis=1, inplace = True)
# df.drop(['MCC_DESCRIPTION'], axis=1, inplace = True)

In [ ]:
df.MCC_DESCRIPTION.value_counts() # 385 MCC_DESCRIPTION

Stationery, Office & School Supply Stores            20691
Book Stores                                          17344
Stationery,Office Supplies,Printing/Writing Paper    14179
Business Services Not Elsewhere Classified           12788
Local/Suburban Commuter Transportation               11067
                                                     ...  
Wig And Toupee Stores                                    1
Bail And Bond Payments                                   1
Tax Payments                                             1
Austrian Airlines                                        1
Precious Stones And Metals, Watches and Jewelry          1
Name: MCC_DESCRIPTION, Length: 385, dtype: int64

In [ ]:
df.AGENCY.value_counts() # 89 Agency

Fire & Emergency Medical Services             19932
District Department of Transportation         18356
District of Columbia Public Schools           15890
Department on Disability Services             13336
Department of Health                          12025
                                              ...  
Advisory Neighborhood Commission                131
Office of Justice Grants Administration         121
Medical Liability Captive Insurance Agency       97
OFFICE OF CAMPAIGN FINANCE                       89
Disability Compensation Fund                     46
Name: AGENCY, Length: 89, dtype: int64

In [ ]:
df.VENDOR_NAME.value_counts() # 39321 vendors

STANDARD OFFICE SUPPLY      12018
AMAZON MKTPLACE PMTS         9591
METRO FARE AUTOLOAD          6232
AMAZON.COM                   5986
METROPOLITAN OFFICE PR       4205
                            ...  
QUICK RESPONSE FIRE SU          1
DELTA     00686706468534        1
ENVIRONMENTAL CAREER C          1
MCDONALD'S F7394                1
WPY*COAST 2 COAST TRAV          1
Name: VENDOR_NAME, Length: 39321, dtype: int64

In [ ]:
df.VENDOR_STATE_PROVINCE = df.VENDOR_STATE_PROVINCE.str.strip() # Remove leading and trailing whitespace
# df.VENDOR_STATE_PROVINCE.value_counts() # 95 VENDOR_STATE_PROVINCE
df.VENDOR_STATE_PROVINCE.unique()

# # States, Provinces, and Regions in North America (https://recruit.smashfly.com/SmashFlyHelp/JobImportAPIHelp/Content/LookupValues/StatesProvincesRegionsNorthAmerica.htm)
# states = ["AK", "AL", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "GU", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MH", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PW", "PA", "PR", "RI", "SC", "SD", "TN", "TX", "UT", "VI", "VT", "VA", "WA", "WV", "WI", "WY", "ZZ", "ON", "QC", "BC", "AB", "MB", "SK", "NS", "NB", "NL", "PE", "NT", "NU", "YT", "AGU", "BCN", "BCS", "CAM", "COL", "CHP", "CHP", "CHH", "DIF", "DUR", "GRO", "GUA", "HID", "JAL", "MEX", "MIC", "MOR", "NAY", "NLE", "OAX", "PUE", "QUE", "ROO", "SIN", "SLP", "SON", "TAB", "TAM", "TLA", "VER", "YUC", "ZAC"]
# set(df.VENDOR_STATE_PROVINCE.unique()) - set(states)

array(['MD', 'MA', 'DC', 'TN', 'IL', 'AR', 'OH', 'FL', 'WV', 'NC', 'CA',
       'VA', 'KY', 'CO', 'NY', 'KS', 'WA', 'AZ', 'WI', 'PA', 'IA', 'MN',
       'UT', 'TX', 'NJ', 'ND', 'GA', 'GBR', 'MI', 'CT', 'LA', nan, 'AUS',
       'NE', 'DE', 'NV', 'NM', 'NH', 'RI', 'MO', 'OK', 'ID', 'IN', 'DEU',
       'SC', 'OR', 'WY', 'ME', 'AL', 'JAM', 'FRA', 'MS', 'QC', 'ON', 'MB',
       'AK', 'VT', 'AB', 'MT', 'BC', 'NLD', '280', 'VIC', 'HKG', 'NS',
       'ESP', 'NSW', 'ILR', 'LLI', 'MYS', 'SD', 'SGP', 'ILP', 'CHE',
       'CYP', 'HI', 'IRL', 'RUS', 'VIR', 'C', 'PR', 'ISR', 'PE', 'ILN',
       '000', 'PRI', 'ILS', 'LUX', 'HK', 'GB', 'MY', 'MX', 'CN', 'IE',
       'AU', 'IO'], dtype=object)

In [ ]:
df['TRANSACTION_DATE'] = pd.to_datetime(df['TRANSACTION_DATE'])
df['TRANSACTION_DATE'] = pd.to_datetime(df.TRANSACTION_DATE).dt.tz_localize(None)

In [ ]:
# # Creating a instance of label Encoder.
# le = LabelEncoder()
 
# # Using .fit_transform function to fit label
# # encoder and return encoded label
# df['MCC_DESCRIPTION_ID'] = le.fit_transform(df['MCC_DESCRIPTION']) # Name: MCC_DESCRIPTION, Length: 385
# df['AGENCY_ID'] = le.fit_transform(df['AGENCY']) # Name: AGENCY, Length: 89
# df['VENDOR_NAME_ID'] = le.fit_transform(df['VENDOR_NAME']) # Name: VENDOR_NAME, Length: 39321
# df['VENDOR_STATE_PROVINCE_ID'] = le.fit_transform(df['VENDOR_STATE_PROVINCE']) # Name: VENDOR_STATE_PROVINCE, Length: 95

# df['MCC_DESCRIPTION_ID'] = df['MCC_DESCRIPTION_ID'].astype(object)
# df['AGENCY_ID'] = df['AGENCY_ID'].astype(object)
# df['VENDOR_NAME_ID'] = df['VENDOR_NAME_ID'].astype(object)
# df['VENDOR_STATE_PROVINCE_ID'] = df['VENDOR_STATE_PROVINCE_ID'].astype(object)
# # printing label
# df.head()

In [ ]:
df.dtypes

AGENCY                           object
TRANSACTION_DATE         datetime64[ns]
TRANSACTION_AMOUNT              float64
VENDOR_NAME                      object
VENDOR_STATE_PROVINCE            object
MCC_DESCRIPTION                  object
dtype: object

## PAR Model
https://sdv.dev/SDV/user_guides/timeseries/par.html

### Sample

In [ ]:
# from sdv.demo import load_timeseries_demo
# data = load_timeseries_demo()
# data

In [ ]:
# data.dtypes

In [ ]:
# entity_columns = ['Symbol']
# context_columns = ['MarketCap', 'Sector', 'Industry'] # the values of the context_columns are expected to remain constant alongside each combination of entity_columns values.
# sequence_index = 'Date'

In [ ]:
# model = PAR(
#             entity_columns=entity_columns,
#             context_columns=context_columns,
#             sequence_index=sequence_index,
#  )
# model.fit(data) # 1m 31s

In [ ]:
# new_data = model.sample(1)
# new_data

### Model

In [ ]:
# range = (df['TRANSACTION_DATE'] > "2009-01-01") & (df['TRANSACTION_DATE'] <= "2010-01-01")
# sample = df.loc[range]
# sample.head() # 95589 rows × 10 columns
sample = df[df['TRANSACTION_DATE'].dt.year==2009]
sample #28235 rows x 6 columns

,AGENCY,TRANSACTION_DATE,TRANSACTION_AMOUNT,VENDOR_NAME,VENDOR_STATE_PROVINCE,MCC_DESCRIPTION
0,Department of Youth Rehabilitation Services,2009-01-06,977.00,MY SAFE FOOD HANDLER,MD,Business Services Not Elsewhere Classified
1,Department of Youth Rehabilitation Services,2009-01-06,-25.61,AMES SAFETY ENVELOPE C,MA,Miscellaneous Publishing And Printing
2,DC Public Library,2009-01-07,1035.29,WWW.AIA.ORG,DC,"Membership Organizations, Not Elsewhere Classi..."
3,Office of the State Superintendent of Education,2009-01-06,222.40,DELTA 00675274330522,TN,Delta
4,Office of the State Superintendent of Education,2009-01-06,222.40,DELTA 00675274330500,TN,Delta
...,...,...,...,...,...,...
42995,Fire & Emergency Medical Services,2009-12-02,1541.48,WW GRAINGER,DC,"Industrial Supplies, Not Elsewhere Classified"
42996,Department of Employment Services,2009-12-02,270.00,DYNAMIC WORKS INSTITUT,FL,"Educational Services, Not Elsewhere Classified"
42997,District Department of Transportation,2009-12-01,440.12,STANDARD OFFICE SUPPLY,DC,"Stationery, Office & School Supply Stores"
42998,District Department of Transportation,2009-12-01,123.22,THE HOME DEPOT #2583,DC,Home Supply Warehouse Stores


In [ ]:
train1 = sample[['TRANSACTION_DATE', 'TRANSACTION_AMOUNT', 'AGENCY', 'VENDOR_STATE_PROVINCE', 'MCC_DESCRIPTION']]
train2 = sample[['TRANSACTION_DATE', 'TRANSACTION_AMOUNT', 'AGENCY', 'VENDOR_STATE_PROVINCE']]
train3 = sample[['TRANSACTION_DATE', 'TRANSACTION_AMOUNT', 'AGENCY']]

In [ ]:
# context_columns = [] # variables that provide information about the entities associated with the timeseries in the form of attributes and which may condition how the timeseries variables evolve.
entity_columns = ['AGENCY'] # These are columns that indicate how the rows are associated with external, abstract, entities.
sequence_index = 'TRANSACTION_DATE'

In [ ]:
model1 = PAR(
            sequence_index=sequence_index,
            # entity_columns=entity_columns,
            # context_columns=context_columns,
            # field_types=field_types,
 )
model2 = PAR(
            sequence_index=sequence_index,
 )
model3 = PAR(
            sequence_index=sequence_index,
 )

In [ ]:
%%time
model3.fit(train3)

CPU times: user 2min 11s, sys: 873 ms, total: 2min 12s
Wall time: 2min 13s


In [ ]:
%%time
synthetic_sample3 = model3.sample(1)

CPU times: user 8min 45s, sys: 4.57 s, total: 8min 50s
Wall time: 8min 50s


In [ ]:
synthetic_sample3

,TRANSACTION_DATE,TRANSACTION_AMOUNT,AGENCY
0,2009-01-02,473.571815,Department of Employment Services
1,2009-01-02,-631.489905,Office of the Attorney General
2,2009-01-02,784.448028,Department of Youth Rehabilitation Services
3,2009-01-02,902.528935,Office of Unified Communications
4,2009-01-02,-356.513609,Commission of Judicial Disabilities & Tenure
...,...,...,...
28230,2009-12-19,-43.000387,Office of Employee Appeals
28231,2009-12-19,955.325900,Department of Housing & Community Development
28232,2009-12-19,580.665878,Department of Consumer & Regulatory Affairs
28233,2009-12-19,507.213091,Department of Youth Rehabilitation Services


In [ ]:
synthetic_sample1.to_csv('par_syn_data.csv', index=False)

In [ ]:
# model1.save('PAR1.pkl')
# synthetic_sample1.to_csv('PAR1_syn.csv')

In [ ]:
# %%time
# model2.fit(train2)

CPU times: user 2min 9s, sys: 1.22 s, total: 2min 10s
Wall time: 2min 9s


In [ ]:
# %%time
# synthetic_sample2 = model2.sample()

CPU times: user 9min 8s, sys: 5.92 s, total: 9min 14s
Wall time: 9min 11s


In [ ]:
# model2.save('PAR2.pkl')
# synthetic_sample2.to_csv('PAR2_syn.csv')

In [ ]:
# %%time
# model3.fit(train3)

CPU times: user 2min 9s, sys: 808 ms, total: 2min 10s
Wall time: 2min 9s


In [ ]:
# %%time
# synthetic_sample3 = model3.sample()

CPU times: user 8min 32s, sys: 5.89 s, total: 8min 38s
Wall time: 8min 36s


In [ ]:
# model3.save('PAR3.pkl')
# synthetic_sample3.to_csv('PAR3_syn.csv')

In [ ]:
# # Load the model and generate new data
# loaded = PAR.load('PAR1.pkl')
# loaded.sample(num_sequences=1).head()